### Create AWS Bedrock Agent Workflow
#### Requirements


In [17]:
import boto3
import os
import json
from dotenv import load_dotenv
load_dotenv()


# Create IAM Role required for Bedrock Agent
session = boto3.Session()
identity = session.client('sts').get_caller_identity()
account_number = identity['Account']
aws_region = os.environ['AWS_REGION']
bedrock_model = os.environ['BEDROCK_MODEL']
bedrock_policy_arn = os.environ['BEDROCK_POLICY_ARN']


# define trust relationship policy document
bedrock_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {
            "Service": "bedrock.amazonaws.com"
        },
        "Action": "sts:AssumeRole",
        "Condition": {
            "StringEquals": {
                "aws:SourceAccount": "196215119328"
            },
            "ArnLike": {
                "AWS:SourceArn": "arn:aws:bedrock:us-east-1:196215119328:agent/*"
            }
        }
    }]
}





# Create IAM Role
iam_client = session.client('iam')
response = iam_client.create_role(
    RoleName='BedrockAgentRole',
    AssumeRolePolicyDocument=json.dumps(bedrock_trust_policy),
    Description='Role for Bedrock Agent',
)
role_arn = response['Role']['Arn']


iam_client.attach_role_policy(
    RoleName='BedrockAgentRole',
    PolicyArn=bedrock_policy_arn,
) 

{'ResponseMetadata': {'RequestId': '5d2c6ed8-85ec-4bab-bd76-88775a0e01a1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 18 Feb 2025 05:57:37 GMT',
   'x-amzn-requestid': '5d2c6ed8-85ec-4bab-bd76-88775a0e01a1',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [ ]:

# Define bedrock agent
bedrock_agent = boto3.client(service_name='bedrock-agent', region_name='us-east-1')
create_agent_response = bedrock_agent.create_agent(
    agentName='event-registration-customer-support',
    foundationModel=bedrock_model,
    agentResourceRoleArn=role_arn,
    instruction=""" You are an advanced AI agent acting as a front line customer support agent for an event. Customers are going to ask you about the event and their registration and stay status. """,
)
agent_id = create_agent_response['agent']['agentId']
print(f'Agent ID: {agent_id}')

In [19]:
# Utility / Helper methods
import time
def agent_status(agent_id: str, target_status: str):
    agent_status = ''
    bedrock_agent = boto3.client(service_name='bedrock-agent', region_name='us-east-1')
    while agent_status != target_status:
        response = bedrock_agent.get_agent(agentId=agent_id)
        agent_status = response['agent']['agentStatus']
        print(f'Agent status: {agent_status}')
        if (agent_status == target_status or agent_status == 'PREPARED'):
            return True
        time.sleep(5)

def agent_alias_status(agent_id: str, agent_alias_id: str, target_status: str):
    agent_alias_status = ''
    bedrock_agent = boto3.client(service_name='bedrock-agent', region_name='us-east-1')
    while agent_alias_status != target_status:
        response = bedrock_agent.get_agent_alias(agentId=agent_id, agentAliasId=agent_alias_id)
        agent_alias_status = response['agentAlias']['agentAliasStatus']
        print(f'Agent Alias status: {agent_alias_status}')
        if (agent_alias_status == target_status or agent_alias_status == 'PREPARED'):
            return True
        time.sleep(5)

In [ ]:
# Check status of agent to become 'CREATING' to 'NOT_PREPARED' 
agent_status(agent_id, 'NOT_PREPARED')

# Once reaches status 'NOT_PREPARED', prepare the agent
bedrock_agent.prepare_agent(agentId=agent_id)

# Check for status of agent to become 'NOT_PREPARED' to 'PREPARED'
agent_status(agent_id, 'PREPARED')

# Once agent is status 'PREPARED', deploy the agent with alias name
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agent_id,
    agentAliasName='Event-Registration-Customer-Support-Alias',
)
agent_alias_id = create_agent_alias_response['agentAlias']['agentAliasId']
print(f'Agent Alias ID: {agent_alias_id}')

# Check for status of agent alias to become 'CREATING' to 'NOT_PREPARED'
agent_alias_status(agent_id, agent_alias_id, 'PREPARED')

